# Maîtriser l’utilisation des serveurs de calcul
## Lots de tâches
Le calcul haute-performance consiste non seulement au calcul parallèle par tâche (***parallélisme des tâches***),
mais aussi au calcul de données en parallèle dans plusieurs tâches et/ou processus en simultané (***parallélisme de données***).
Ce chapitre vous donnera les outils nécessaires pour gérer un grand nombre de tâches
lorsque le projet de recherche requiert plusieurs centaines de résultats.

## GNU Parallel
La commande `parallel` de [GNU Parallel](https://docs.computecanada.ca/wiki/GNU_Parallel/fr) permet d'utiliser pleinement les ressources locales d'un noeud de calcul, et ce, en gérant l'exécution d'une **longue liste de tâches de *petite* taille**.
C'est un peu comme l'ordonnanceur Slurm, mais à plus petite échelle et en gérant des processus au lieu de scripts de tâche.
* [Documentation officielle](https://www.gnu.org/software/parallel/parallel.html)
* [Tutoriel](https://www.gnu.org/software/parallel/parallel_tutorial.html)

### Pourquoi pas Slurm?
OK, mais pourquoi ne pas tout simplement soumettre **des centaines de tâches à Slurm**?
* À tout moment, Slurm **limite chaque usager à 1000 tâches** en tout dans `squeue` (*pending* + *running*)
* Certains calculs sont tellement **courts (< 5 minutes)** que le démarrage et la fin de la tâche Slurm compteraient pour un pourcentage significatif du temps réel utilisé
  * On veut donc réduire les pertes de temps
* GNU Parallel nous **évite d'utiliser un pipeline** soumettant des centaines de scripts similaires, ce qui, dans bien des cas, facilite l'exécution de centaines de tâches de calcul semblables
* Enfin, si toutes vos tâches envoient des **notifications par courriel**, imaginez recevoir ces centaines de courriels!

Les autres avantages à considérer :
* Par défaut, le nombre de **processeurs disponibles limite** le nombre de tâches de calcul exécutées en simultané
  * Dans le cas de calculs parallèles, c'est possible de limiter davantage le nombre de processus
* GNU Parallel peut [reprendre la séquence des tâches de calcul](https://docs.computecanada.ca/wiki/GNU_Parallel/fr#Suivi_des_commandes_ex.C3.A9cut.C3.A9es_ou_des_commandes_ayant_.C3.A9chou.C3.A9.3B_fonctionnalit.C3.A9s_de_red.C3.A9marrage) en cas de fin hâtive de la tâche Slurm

### Syntaxe de la commande GNU Parallel
Les éléments de base de la commande `parallel` : options, gabarit de commande, liste de valeurs. Voir :
```
man parallel
```

### Cas d'utilisation
#### Séquence de paramètre
Le paramètre changeant est donné via une paire d'`{}` :
```
parallel echo fichier{}.txt ::: 1 2 3 4
parallel --citation  # Il faut citer les développeurs
```

Utilisation d'une séquence générée par la commande `seq` :
```
seq 1 20
seq -w 1 20
echo $(seq -w 1 20)
parallel echo fichier{}.txt ::: $(seq -w 1 20)
```

#### Combinaisons de paramètres
Les paramètres changeants peuvent être donnés via une paire d'`{}`, ou plus précisément par des paire d'`{}` numérotées telles que `{1}`, `{2}`, etc. :
```
parallel echo fichier{}.txt ::: $(seq -w 1 10) ::: a b
parallel echo fichier{1}{2}.txt ::: $(seq -w 1 10) ::: a b
```

Si jamais les différentes combinaisons de paramètres proviennent d'un fichier texte :
```
parallel echo {} ::: 3 5 7 ::: 4 6 8 > param.txt
cat param.txt
```
Par la suite, dans le script de tâche Slurm :
```
parallel -C ' ' echo '{1}*{2} | bc > prod_{1}x{2}' :::: param.txt
ls prod_*
grep -E '[0-9]+' prod*
```

Si on préfère valider la liste des commandes générées avant de soumettre une tâche Slurm, on peut utiliser `parallel` pour la générer dans un fichier. Par exemple :
```
parallel -C ' ' echo 'echo {1}"*"{2} "|" bc ">" prod_{1}x{2}' :::: param.txt > commandes.txt
cat commandes.txt
```
Et dans la tâche Slurm :
```
parallel < commandes.txt
grep -E '[0-9]+' prod_*
```

#### Limiter le nombre de tâches parallèles
Par exemple, sur 8 processeurs, c'est possible de limiter le nombre de tâches à deux (2) tâches parallèles de quatre (4) processeurs chacune :
```
parallel --jobs 2 'echo {} && sleep 3' ::: $(seq -w 1 8)
```

Si jamais un noeud n'est pas suffisant pour répondre au besoin de centaines de tâches parallèles, GNU Parallel peut distribuer des tâches sur plusieurs noeuds
```
salloc --nodes=2 --ntasks-per-node=1 --mem-per-cpu=2000M
  scontrol show hostname > noeuds.txt
  cat noeuds.txt
  parallel --jobs $SLURM_NTASKS_PER_NODE --sshloginfile noeuds.txt \
    'echo -n {}: && hostname' ::: $(seq 1 $SLURM_NTASKS)
```

### Exercice - Paramètres de Benchmark5D
Dans cet exercice, il est question d'une recherche de paramètres optimaux pour un code "quelconque".
Ce code utilise $O(n^3)$ valeurs numériques pour $O(n^5)$ opérations à double précision.
En vous épargnant les détails du programme, les différents paramètres à tester permettent d'analyser son comportement à l'exécution.
En bref, de petites valeurs de `p*` et/ou `b*` forcent le programme à calculer le plus localement possible dans la mémoire-vive, ce qui devrait donner une meilleure performance.

Pour installer le programme Benchmark5D :
```
bash scripts/installer/b5D.sh
```

Pour lancer une tâche de 4 processeurs utilisant GNU Parallel pour tester 3*4 combinaisons de paramètres :
```
cat scripts/b5D-parallel.sh
sbatch scripts/b5D-parallel.sh
```

Pour le suivi de la tâche :
```
squeue -u $USER
```
Les résultats seront dans un fichier `slurm-<jobid>.out`
```
cat slurm-<jobid>.out
```
À la fin de la tâche, on peut analyser les ressources utilisées :
```
seff <jobid>
```

## Vecteurs de tâches
Alors que GNU Parallel s'utilise bien avec des calculs séquentiels et parallèles de petite taille (<32 processeurs),
GNU Parallel n'est pas vraiment l'outil approprié pour un lot de longs calculs parallèles de plus grande taille (>=32 processeurs par calcul).
Dans le cas où un même programme parallèle doit être exécuté avec différentes combinaisons de paramètres,
il y a moyen de soumettre un seul [vecteur de tâches](https://docs.computecanada.ca/wiki/Job_arrays/fr) et de coder le script de tâche de telle sorte que les paramètres seront déterminés en fonction d'un indice unique du vecteur de tâches.

Pour soumettre un vecteur de tâches à l'ordonnanceur Slurm, à la ligne de commande `sbatch` ou dans l'entête `#SBATCH` du script de tâche, on doit ajouter l'option `--array=<indices>`.
Voir [ici quelques exemples](https://docs.computecanada.ca/wiki/Job_arrays/fr).

L'identifiant d'une tâche Slurm dans un vecteur de tâches contient :
* L'identifiant du vecteur de tâches
* Caractère de soulignement (`_`)
* L'indice unique associé à la tâche

Par exemple : `25249551_15`

Dans le script de tâche, la variable d'environnement `$SLURM_ARRAY_TASK_ID` peut être utilisée pour retrouver la valeur actuelle de l'indice unique associé à la tâche en cours d'exécution, soit un entier parmis les `<indices>`.
La variable `$SLURM_ARRAY_TASK_ID` peut être utilisée de différentes façons :
* `nom_fichier.$SLURM_ARRAY_TASK_ID`
* `nom_dossier-$SLURM_ARRAY_TASK_ID`
* `PARAM=$SLURM_ARRAY_TASK_ID`
* `PARAM1=$((SLURM_ARRAY_TASK_ID / 12))`
* `PARAM2=$((SLURM_ARRAY_TASK_ID % 12))`
* `PARAMS=$(sed -n "${SLURM_ARRAY_TASK_ID}p" combinaisons.txt)`

### Exercice - Vecteur de tâches
Pour lancer le vecteur de tâches :
* `sbatch scripts/executer-sleep.sh`

Après l'exécution des trois tâches, on peut inspecter les résultats :
* `cat slurm-*_*.out`

## Le *Greedy Launcher Of Small Tasks*, ou GLOST
Le *Greedy Launcher Of Small Tasks*, ou [GLOST](https://docs.computecanada.ca/wiki/GLOST/fr), fonctionne un peu comme GNU Parallel, mais avec quelques différences :
* Fonctionne qu'avec des tâches séquentielles (un processeur par tâche)
* L'ordonnancement des processus se fait via [MPI](https://docs.computecanada.ca/wiki/MPI/fr) avec une architecture de type gestionnaire-travailleur
  * Le gestionnaire envoie à chaque travailleur une tâche séquentielle à exécuter
  * Le travailleur signale au gestionnaire quand le travail est complété
  * Le gestionnaire informe les travailleurs lorsqu'il n'y a plus de tâche (pour quitter)
* Chaque tâche est définie comme une ligne de commande
  * Toutes les lignes de commande sont [listées dans un même fichier](https://docs.computecanada.ca/wiki/GLOST/fr#Liste_de_t.C3.A2ches_situ.C3.A9es_dans_le_m.C3.AAme_r.C3.A9pertoire). Par exemple : `list_glost_tasks.txt`
  * Les sous-commandes peuvent être séparées par l'opérateur `&&`. Par exemple : `echo -n 'Bonjour ' && echo le monde`

### Exercice - Liste de tâches Benchmark5D avec GLOST
Pour générer la liste de tâches dans le fichier `$SCRATCH/cas-b5D.txt` :
* `bash scripts/gen-cas-b5D.sh`
* `cat $SCRATCH/cas-b5D.txt`

Pour lancer une tâche de 4 processeurs utilisant GLOST pour tester 3*4 combinaisons de paramètres :
* `sbatch scripts/glost-b5D.sh`

Pour le suivi de la tâche :
* `squeue -u $USER`
* Les résultats seront dans un fichier `slurm-<jobid>.out`
  * `cat slurm-<jobid>.out`
* À la fin de la tâche, on peut analyser les ressources utilisées :
  * `seff <jobid>`